In [1]:
class NetworkNode:
    def __init__(self, name):
        self.name = name  # Name of the node
        self.neighbors = {}  # Dictionary of neighbors and their costs
        self.distance_vector = {}  # Distance vector table (destination: cost)
        self.next_hop = {}  # Next hop table (destination: next hop)

    def add_neighbor(self, neighbor, cost):
        self.neighbors[neighbor] = cost
        self.distance_vector[neighbor] = cost
        self.next_hop[neighbor] = neighbor

    def remove_neighbor(self, neighbor):
        if neighbor in self.neighbors:
            del self.neighbors[neighbor]
            if neighbor in self.distance_vector:
                del self.distance_vector[neighbor]
                del self.next_hop[neighbor]

    def update_distance_vector(self, all_nodes):
        updated = False
        for neighbor, neighbor_cost in self.neighbors.items():
            for destination, cost_via_neighbor in all_nodes[neighbor].distance_vector.items():
                if destination == self.name:
                    continue  # Skip self-reference
                new_cost = neighbor_cost + cost_via_neighbor
                if destination not in self.distance_vector or new_cost < self.distance_vector[destination]:
                    self.distance_vector[destination] = new_cost
                    self.next_hop[destination] = neighbor
                    updated = True

        # Add self to the distance vector with cost 0
        if self.name not in self.distance_vector:
            self.distance_vector[self.name] = 0
            self.next_hop[self.name] = self.name

        return updated

    def __str__(self):
        output = f"Distance vector for {self.name}:\n"
        for destination, cost in self.distance_vector.items():
            if destination == self.name:
                continue  # Skip self-reference
            output += f"  Destination {destination}: Cost = {cost}, Next hop = {self.next_hop[destination]}\n"
        return output

def distance_vector_routing_simulation(nodes, links):
    # Create a dictionary of all nodes
    all_nodes = {name: NetworkNode(name) for name in nodes}

    # Add neighbors based on the links
    for node1, node2, cost in links:
        all_nodes[node1].add_neighbor(node2, cost)
        all_nodes[node2].add_neighbor(node1, cost)

    # Update distance vectors until convergence
    converged = False
    while not converged:
        converged = True
        for node in all_nodes.values():
            if node.update_distance_vector(all_nodes):
                converged = False

    # Print the distance vectors
    for node in all_nodes.values():
        print(node)

    return all_nodes

def add_link(node1, node2, cost, all_nodes):
    """Add a new link between two nodes."""
    if node1 in all_nodes and node2 in all_nodes:
        all_nodes[node1].add_neighbor(node2, cost)
        all_nodes[node2].add_neighbor(node1, cost)
        print(f"Link added between {node1} and {node2} with cost {cost}.")
    else:
        print("Error: One or both nodes do not exist.")

def remove_link(node1, node2, all_nodes):
    """Remove an existing link between two nodes."""
    if node1 in all_nodes and node2 in all_nodes:
        all_nodes[node1].remove_neighbor(node2)
        all_nodes[node2].remove_neighbor(node1)
        print(f"Link removed between {node1} and {node2}.")
    else:
        print("Error: One or both nodes do not exist.")

def simulate_traffic(source, destination, all_nodes):
    """Simulate traffic to verify the shortest path between two nodes."""
    if source in all_nodes and destination in all_nodes:
        path = []
        current = source
        total_cost = 0
        while current != destination:
            path.append(current)
            if destination in all_nodes[current].next_hop:
                next_hop = all_nodes[current].next_hop[destination]
                total_cost += all_nodes[current].neighbors[next_hop]
                current = next_hop
            else:
                print("No path found.")
                return
        path.append(destination)
        print(f"Shortest path from {source} to {destination}: {' -> '.join(path)} with total cost {total_cost}.")
    else:
        print("Error: One or both nodes do not exist.")

# Example usage:
nodes = ['A', 'B', 'C', 'D']
links = [
    ('A', 'B', 1),
    ('A', 'C', 4),
    ('B', 'C', 2),
    ('B', 'D', 6),
    ('C', 'D', 3)
]

all_nodes = distance_vector_routing_simulation(nodes, links)

# Add a new link
add_link('A', 'D', 5, all_nodes)

# Remove an existing link
remove_link('A', 'C', all_nodes)

# Simulate traffic
simulate_traffic('A', 'D', all_nodes)

# Re-run the simulation to update routing tables after changes
print("\nRecomputing distance vectors after link updates:\n")
distance_vector_routing_simulation(nodes, links)


Distance vector for A:
  Destination B: Cost = 1, Next hop = B
  Destination C: Cost = 3, Next hop = B
  Destination D: Cost = 6, Next hop = B

Distance vector for B:
  Destination A: Cost = 1, Next hop = A
  Destination C: Cost = 2, Next hop = C
  Destination D: Cost = 5, Next hop = C

Distance vector for C:
  Destination A: Cost = 3, Next hop = B
  Destination B: Cost = 2, Next hop = B
  Destination D: Cost = 3, Next hop = D

Distance vector for D:
  Destination B: Cost = 5, Next hop = C
  Destination C: Cost = 3, Next hop = C
  Destination A: Cost = 6, Next hop = C

Link added between A and D with cost 5.
Link removed between A and C.
Shortest path from A to D: A -> D with total cost 5.

Recomputing distance vectors after link updates:

Distance vector for A:
  Destination B: Cost = 1, Next hop = B
  Destination C: Cost = 3, Next hop = B
  Destination D: Cost = 6, Next hop = B

Distance vector for B:
  Destination A: Cost = 1, Next hop = A
  Destination C: Cost = 2, Next hop = C
  D

{'A': <__main__.NetworkNode at 0x1f6a05caf30>,
 'B': <__main__.NetworkNode at 0x1f6a05caff0>,
 'C': <__main__.NetworkNode at 0x1f6a05cb0b0>,
 'D': <__main__.NetworkNode at 0x1f6a05cb1a0>}